# Creating Your First Aurora Cluster - Part 1

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#What-We'll-Build">What We'll Build</a></li>
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Cost-Overview">Cost Overview 💰</a></li>
<li><a href="#Best-Practices">Best Practices ✅</a></li>
<li><a href="#Step-1-Create-VPC-and-Networking">Step 1: Create VPC and Networking</a></li>
<li><a href="#Step-2-Security-Group-Configuration">Step 2: Security Group Configuration</a></li>
<li><a href="#Step-3-Create-S3-VPC-Endpoint">Step 3: Create S3 VPC Endpoint</a></li>
<li><a href="#Next-Steps">Next Steps 🚀</a></li>
</ul>
</div>

Let's create a production-ready Aurora PostgreSQL cluster step by step! 🚀

> **💡 Quick Start Available**: Want to skip the manual setup on **networking and database creation**? Launch the infrastructure with our [AWS CloudFormation template](../cfn/aurora-complete-stack.yml). Then, feel free to explore from the [AWS console](https://console.aws.amazon.com/rds) for what you have created.

> **💡 Note**: **If you are using [the Amazon Sagemaker Notebook launched during setting up your cookbook environment](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD), the network resources have been created already**. You can [follow the steps for creating your Amazon Aurora cluster](./2.1.2_create_your_first_aurora_postgresql_part2.ipynb). For practice, feel free to test with the scripts in this notebook.

## What We'll Build

- **Network Setup**: VPC with public/private subnets across 2 AZs, routing, private subnet isolation, and security groups
- **Database Layer**: Aurora PostgreSQL Serverless v2 cluster with writer and reader instances

## Prerequisites

Before starting this workshop, ensure you have:

- ✅ **AWS Account**: Active AWS account with appropriate permissions
  - [Create your first AWS account](../1_Getting_Started_with_AWS/README.MD) if you have not
- ✅ **Jupyter Environment**: This notebook should be run in:
  - [Amazon SageMaker Jupyter notebook (recommended)](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
  - Or, local Jupyter with AWS CLI configured. Ensure [database access from your local Jupyter](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- ✅ **IAM Permissions** for:
  - VPC and networking resources (EC2 full access)
  - RDS cluster creation and management
  - S3 VPC endpoint creation
- ✅ **AWS CLI**: Configured with credentials and default region
  - [Amazon SageMaker Jupyter notebook (recommended)](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD) has an IAM role with permissions to access relevant resources

> 💡 **Note**: This notebook creates the networking foundation. You'll create the actual Aurora cluster in [Part 2](./2.1.2_create_your_first_aurora_postgresql_part2.ipynb).

## Cost Overview 💰

Use [AWS Pricing Calculator](https://calculator.aws/#/) to estimate the cost for your architecture solution. The following a rough cost estimation for the resources created in this notebook.

| Component | Cost (us-east-1) | Notes |
|-----------|------------------|--------|
| VPC | Free | No charge for creating and using a VPC |
| Subnets | Free | No charge for creating subnets |
| Internet Gateway | Free | No charge for Internet Gateway |
| Route Tables | Free | No charge for route tables and routes |
| Security Groups | Free | No charge for security group rules |
| S3 VPC Gateway Endpoint | Free | [No additional charge for using gateway endpoints](https://docs.aws.amazon.com/vpc/latest/privatelink/vpc-endpoints-s3.html) |

💡 **Cost Optimization Tips:**
- The networking components created in this part are all free of charge
- In [Part 2 Creating Your First Aurora Cluster](./2.1.2_create_your_first_aurora_postgresql_part2.ipynb), when creating the Amazon Aurora database, consider:
  - Using auto-pause feature to allow Amazon Aurora servelerss cluster scaling down to 0 ACU
  - Setting appropriate min/max ACU (Aurora Compute Unit) values based on workload
  - Using standard storage instead of I/O optimized for non-intensive workloads

## Best Practices ✅

1. **Network Design**
   - Always place databases in private subnets
   - Use multiple AZs for high availability
   - Implement proper CIDR planning for future growth

2. **Security**
   - Enable encryption at rest
   - Use security groups for fine-grained access control
   - Implement proper IAM roles and policies

Let's get started! 🚀

## Step 1: Create VPC and Networking

### Why This Step Is Important
A well-designed VPC is crucial for security and scalability. We're creating:
- VPC with 10.0.0.0/16 CIDR (65,536 IP addresses)
- Public subnets for external access
- Private subnets for Aurora (isolated from internet)
- Multiple AZs for high availability

### Common Pitfalls to Avoid
- ❌ Using overlapping CIDR ranges
- ❌ Putting databases in public subnets
- ❌ Not enabling DNS hostnames in VPC
- ❌ Using too small CIDR ranges

### Troubleshooting Guide
If you encounter errors:
1. **'VPC limit exceeded'**: Delete unused VPCs or request limit increase
2. **'CIDR conflict'**: Ensure CIDR doesn't overlap with other VPCs
3. **'AZ not available'**: Try different AZ or region

Let's create our network infrastructure:

### 1. Create VPC

First, let's create a VPC with DNS support enabled. When successful, you'll see a VPC ID in the format `vpc-xxxxxxxxxxxxxxxxx`.

**Key Settings:**
- CIDR Block: 10.0.0.0/16 (65,536 IP addresses)
- DNS hostname support: Enabled
- DNS resolution: Enabled

In [ ]:
%%bash
echo "Creating VPC..."
VPC_ID=$(aws ec2 create-vpc \
  --cidr-block 10.0.0.0/16 \
  --tag-specifications 'ResourceType=vpc,Tags=[{Key=Name,Value=aurora-vpc}]' \
  --query 'Vpc.VpcId' \
  --output text)

# Enable DNS support and hostnames separately
aws ec2 modify-vpc-attribute --vpc-id $VPC_ID --enable-dns-support
aws ec2 modify-vpc-attribute --vpc-id $VPC_ID --enable-dns-hostnames

if [ ! -z "$VPC_ID" ]; then
    echo "✅ VPC created successfully: $VPC_ID"
    echo "Expected output format: vpc-xxxxxxxxxxxxxxxxx"
    # Save VPC_ID for use in other cells
    echo "export VPC_ID=$VPC_ID" > .vpc_vars
else
    echo "❌ VPC creation failed"
    exit 1
fi

### 2. Create Public Subnets

Now we'll create two public subnets in different AZs. These can be used for resources that need internet access.

**Subnet Details:**
- Public Subnet A: 10.0.0.0/24 (first available AZ)
- Public Subnet B: 10.0.1.0/24 (second available AZ)

In [ ]:
%%bash
# Source VPC variables from previous step
source .vpc_vars

# Get available AZs for current region
echo "Getting available AZs for current region..."
AZ_A=$(aws ec2 describe-availability-zones --query 'AvailabilityZones[0].ZoneName' --output text)
AZ_B=$(aws ec2 describe-availability-zones --query 'AvailabilityZones[1].ZoneName' --output text)

echo "Using AZs: $AZ_A and $AZ_B"

# Create public subnet in first AZ
echo "Creating public subnet in $AZ_A..."
PUBLIC_SUBNET_A=$(aws ec2 create-subnet \
  --vpc-id $VPC_ID \
  --cidr-block 10.0.0.0/24 \
  --availability-zone $AZ_A \
  --tag-specifications "ResourceType=subnet,Tags=[{Key=Name,Value=aurora-public-${AZ_A: -1}}]" \
  --query 'Subnet.SubnetId' \
  --output text)

if [ ! -z "$PUBLIC_SUBNET_A" ]; then
    echo "✅ Public subnet in $AZ_A created successfully: $PUBLIC_SUBNET_A"
else
    echo "❌ Public subnet in $AZ_A creation failed"
    exit 1
fi

# Create public subnet in second AZ
echo "Creating public subnet in $AZ_B..."
PUBLIC_SUBNET_B=$(aws ec2 create-subnet \
  --vpc-id $VPC_ID \
  --cidr-block 10.0.1.0/24 \
  --availability-zone $AZ_B \
  --tag-specifications "ResourceType=subnet,Tags=[{Key=Name,Value=aurora-public-${AZ_B: -1}}]" \
  --query 'Subnet.SubnetId' \
  --output text)

if [ ! -z "$PUBLIC_SUBNET_B" ]; then
    echo "✅ Public subnet in $AZ_B created successfully: $PUBLIC_SUBNET_B"
    # Save subnet IDs and AZs for use in other cells
    echo "export PUBLIC_SUBNET_A=$PUBLIC_SUBNET_A" >> .vpc_vars
    echo "export PUBLIC_SUBNET_B=$PUBLIC_SUBNET_B" >> .vpc_vars
    echo "export AZ_A=$AZ_A" >> .vpc_vars
    echo "export AZ_B=$AZ_B" >> .vpc_vars
else
    echo "❌ Public subnet in $AZ_B creation failed"
    exit 1
fi

### 3. Configure Public Subnets

Enable auto-assign public IPv4 addresses for the public subnets. This is required for resources in these subnets to have public IP addresses automatically assigned.

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

echo "Configuring public subnets..."
aws ec2 modify-subnet-attribute \
  --subnet-id $PUBLIC_SUBNET_A \
  --map-public-ip-on-launch

aws ec2 modify-subnet-attribute \
  --subnet-id $PUBLIC_SUBNET_B \
  --map-public-ip-on-launch

echo "✅ Public subnets configured successfully"

### 4. Create Private Subnets

Next, we'll create two private subnets for our Aurora database cluster.

**Subnet Details:**
- Private Subnet A: 10.0.2.0/24 (first available AZ)
- Private Subnet B: 10.0.3.0/24 (second available AZ)

These subnets will host our Aurora instances in a secure, isolated environment.

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

# Create private subnet in first AZ
echo "Creating private subnet in $AZ_A..."
PRIVATE_SUBNET_A=$(aws ec2 create-subnet \
  --vpc-id $VPC_ID \
  --cidr-block 10.0.2.0/24 \
  --availability-zone $AZ_A \
  --tag-specifications "ResourceType=subnet,Tags=[{Key=Name,Value=aurora-private-${AZ_A: -1}}]" \
  --query 'Subnet.SubnetId' \
  --output text)

if [ ! -z "$PRIVATE_SUBNET_A" ]; then
    echo "✅ Private subnet in $AZ_A created successfully: $PRIVATE_SUBNET_A"
else
    echo "❌ Private subnet in $AZ_A creation failed"
    exit 1
fi

# Create private subnet in second AZ
echo "\nCreating private subnet in $AZ_B..."
PRIVATE_SUBNET_B=$(aws ec2 create-subnet \
  --vpc-id $VPC_ID \
  --cidr-block 10.0.3.0/24 \
  --availability-zone $AZ_B \
  --tag-specifications "ResourceType=subnet,Tags=[{Key=Name,Value=aurora-private-${AZ_B: -1}}]" \
  --query 'Subnet.SubnetId' \
  --output text)

if [ ! -z "$PRIVATE_SUBNET_B" ]; then
    echo "✅ Private subnet in $AZ_B created successfully: $PRIVATE_SUBNET_B"
    # Save private subnet IDs for use in other cells
    echo "export PRIVATE_SUBNET_A=$PRIVATE_SUBNET_A" >> .vpc_vars
    echo "export PRIVATE_SUBNET_B=$PRIVATE_SUBNET_B" >> .vpc_vars
else
    echo "❌ Private subnet in $AZ_B creation failed"
    exit 1
fi

### 5. Create Internet Gateway

Now, we'll create an Internet Gateway and attach it to our VPC. This enables internet connectivity for our public subnets.

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

echo "Creating Internet Gateway..."
IGW_ID=$(aws ec2 create-internet-gateway \
    --tag-specifications 'ResourceType=internet-gateway,Tags=[{Key=Name,Value=aurora-igw}]' \
    --query 'InternetGateway.InternetGatewayId' \
    --output text)

if [ ! -z "$IGW_ID" ]; then
    echo "✅ Internet Gateway created successfully: $IGW_ID"
    
    echo "Attaching Internet Gateway to VPC..."
    aws ec2 attach-internet-gateway \
        --vpc-id $VPC_ID \
        --internet-gateway-id $IGW_ID
    echo "✅ Internet Gateway attached to VPC"
    # Save IGW ID for use in other cells
    echo "export IGW_ID=$IGW_ID" >> .vpc_vars
else
    echo "❌ Internet Gateway creation failed"
    exit 1
fi

### 6. Create Route Tables

We'll create two route tables:
- Public route table (routes internet traffic through IGW)
- Private route table (isolated from internet)

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

# Create public route table
echo "Creating public route table..."
PUBLIC_RTB=$(aws ec2 create-route-table \
    --vpc-id $VPC_ID \
    --tag-specifications 'ResourceType=route-table,Tags=[{Key=Name,Value=aurora-public-rtb}]' \
    --query 'RouteTable.RouteTableId' \
    --output text)

# Create private route table
echo "Creating private route table..."
PRIVATE_RTB=$(aws ec2 create-route-table \
    --vpc-id $VPC_ID \
    --tag-specifications 'ResourceType=route-table,Tags=[{Key=Name,Value=aurora-private-rtb}]' \
    --query 'RouteTable.RouteTableId' \
    --output text)

if [ ! -z "$PUBLIC_RTB" ] && [ ! -z "$PRIVATE_RTB" ]; then
    echo "✅ Route tables created successfully:"
    echo "Public Route Table: $PUBLIC_RTB"
    echo "Private Route Table: $PRIVATE_RTB"
    # Save route table IDs for use in other cells
    echo "export PUBLIC_RTB=$PUBLIC_RTB" >> .vpc_vars
    echo "export PRIVATE_RTB=$PRIVATE_RTB" >> .vpc_vars
else
    echo "❌ Route table creation failed"
    exit 1
fi

### 7. Configure Routes

Now we'll add routes to our route tables:
- Public route table: Route internet traffic (0.0.0.0/0) through IGW
- Private route table: No internet access

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

# Add routes
echo "Adding routes to route tables..."

# Public route through Internet Gateway
aws ec2 create-route \
    --route-table-id $PUBLIC_RTB \
    --destination-cidr-block 0.0.0.0/0 \
    --gateway-id $IGW_ID

echo "✅ Routes added successfully"

### 8. Associate Subnets with Route Tables

Finally, we'll associate our subnets with the appropriate route tables:
- Public subnets → Public route table
- Private subnets → Private route table

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

echo "Associating subnets with route tables..."

# Associate public subnets
aws ec2 associate-route-table \
    --subnet-id $PUBLIC_SUBNET_A \
    --route-table-id $PUBLIC_RTB

aws ec2 associate-route-table \
    --subnet-id $PUBLIC_SUBNET_B \
    --route-table-id $PUBLIC_RTB

# Associate private subnets
aws ec2 associate-route-table \
    --subnet-id $PRIVATE_SUBNET_A \
    --route-table-id $PRIVATE_RTB

aws ec2 associate-route-table \
    --subnet-id $PRIVATE_SUBNET_B \
    --route-table-id $PRIVATE_RTB

echo "✅ Subnet associations completed"

### 9. Verify Network Configuration

Let's verify all our network components are properly configured. You should see:
- 4 subnets (2 public, 2 private)
- Correct CIDR ranges
- Proper AZ distribution
- Public IP mapping enabled for public subnets

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

echo "🔍 Verifying subnet configuration..."
aws ec2 describe-subnets \
  --filters "Name=vpc-id,Values=$VPC_ID" \
  --query 'Subnets[*].{SubnetId:SubnetId,CIDR:CidrBlock,AZ:AvailabilityZone,Public:MapPublicIpOnLaunch}' \
  --output table

echo "🔍 Verifying route tables and routes..."
aws ec2 describe-route-tables \
    --filters "Name=vpc-id,Values=$VPC_ID" \
    --query 'RouteTables[*].{ID:RouteTableId,Routes:Routes[*].{Destination:DestinationCidrBlock,Target:GatewayId||NatGatewayId}}' \
    --output table

echo "✅ Network Configuration Summary:"
echo "VPC: $VPC_ID"
echo "Public Subnets: $PUBLIC_SUBNET_A ($AZ_A), $PUBLIC_SUBNET_B ($AZ_B)"
echo "Private Subnets: $PRIVATE_SUBNET_A ($AZ_A), $PRIVATE_SUBNET_B ($AZ_B)"
echo "All subnets created successfully with proper CIDR ranges and AZ distribution."

## Step 2: Security Group Configuration

### Security Best Practices ✅
1. **Principle of Least Privilege**
   - Allow only necessary ports (5432 for PostgreSQL)
   - Restrict source IPs to known application subnets
   - Regular audit of security group rules

2. **Network Isolation**
   - No direct internet access to database
   - Use separate security groups for different tiers
   - Document all rule changes

### Common Security Pitfalls
- ❌ Opening database port to 0.0.0.0/0
- ❌ Using overly permissive rules
- ❌ Not documenting security group changes
- ❌ Mixing different application tiers in same security group

This comprehensive script creates a security group specifically for Aurora database access with properly configured inbound rules and resource tagging. The security group allows PostgreSQL traffic (port 5432) from within the VPC CIDR range while maintaining security isolation from external networks. You'll see the security group ID and rule verification output confirming successful creation.

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

# Create security group
SG_ID=$(aws ec2 create-security-group \
    --group-name aurora-sg \
    --description "Security group for Aurora cluster" \
    --vpc-id $VPC_ID \
    --tag-specifications 'ResourceType=security-group,Tags=[{Key=Name,Value=aurora-sg}]' \
    --query 'GroupId' \
    --output text)

# Add inbound rule for PostgreSQL from private subnets only
aws ec2 authorize-security-group-ingress \
    --group-id $SG_ID \
    --protocol tcp \
    --port 5432 \
    --cidr 10.0.0.0/16  # VPC CIDR range

# Tag the security group for better resource management
aws ec2 create-tags \
    --resources $SG_ID \
    --tags Key=Environment,Value=Production \
           Key=Purpose,Value=AuroraDatabase

# Verify security group rules
aws ec2 describe-security-group-rules \
    --filters Name=group-id,Values=$SG_ID \
    --query 'SecurityGroupRules[*].{Type:IsEgress,Protocol:IpProtocol,Port:FromPort,Source:CidrIpv4}'

echo "✅ Security group created and configured: $SG_ID"
# Save security group ID for use in other cells
echo "export SG_ID=$SG_ID" >> .vpc_vars

## Step 3: Create S3 VPC Endpoint

To allow Aurora PostgreSQL to load data from S3 without going through the public internet, we'll create an S3 VPC endpoint. This provides a secure connection between your VPC and S3.

### Why S3 Endpoint Is Important
- Enables Aurora to import/export data directly from/to S3
- Keeps traffic within AWS network (doesn't traverse the internet)
- Improves data transfer performance
- Enhances security by eliminating public internet exposure

This command creates an S3 VPC endpoint and associates it with the private route table to enable secure data transfer between Aurora and S3 without internet routing. The endpoint configuration includes proper tagging and verification steps to ensure successful setup. You'll see the endpoint ID, service details, and routing table associations in the output.

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

# Create S3 VPC Endpoint
echo "Creating S3 VPC Endpoint..."

S3_ENDPOINT_ID=$(aws ec2 create-vpc-endpoint \
    --vpc-id $VPC_ID \
    --service-name com.amazonaws.$(aws configure get region).s3 \
    --route-table-ids $PRIVATE_RTB \
    --tag-specifications 'ResourceType=vpc-endpoint,Tags=[{Key=Name,Value=aurora-s3-endpoint}]' \
    --query 'VpcEndpoint.VpcEndpointId' \
    --output text)

if [ ! -z "$S3_ENDPOINT_ID" ]; then
    echo "✅ S3 VPC Endpoint created successfully: $S3_ENDPOINT_ID"
else
    echo "❌ S3 VPC Endpoint creation failed"
    exit 1
fi

# Verify the endpoint
echo "🔍 Verifying S3 VPC Endpoint..."
aws ec2 describe-vpc-endpoints \
    --vpc-endpoint-ids $S3_ENDPOINT_ID \
    --query 'VpcEndpoints[0].{ID:VpcEndpointId,Service:ServiceName,State:State,RouteTableIds:RouteTableIds}' \
    --output table

### Using the S3 Endpoint with Aurora PostgreSQL

Once your Aurora PostgreSQL cluster is created, you can use the `aws_s3` extension to import/export data directly to/from S3 buckets. Here's a preview of how you'll use it:

```sql
-- Enable the aws_s3 extension
CREATE EXTENSION IF NOT EXISTS aws_s3;

-- Import data from S3
SELECT aws_s3.table_import_from_s3(
  'target_table',
  '',
  'DELIMITER ''\t''',
  'bucket-name',
  'sample-data.csv',
  'us-east-1'
);

-- Export data to S3
SELECT aws_s3.query_export_to_s3(
  'SELECT * FROM target_table',
  '',
  'DELIMITER ''\t''',
  'bucket-name',
  'exported-data.csv',
  'us-east-1'
);
```

The S3 endpoint we just created ensures these operations are performed securely without traversing the public internet. Using the following script, you'll see protocol details, port configurations, and CIDR restrictions confirming proper security setup.

In [ ]:
%%bash
# Source variables from previous steps
source .vpc_vars

# Check security group details
aws ec2 describe-security-groups \
    --group-ids $SG_ID \
    --output table \
    --query 'SecurityGroups[0].{"Group ID":GroupId,"VPC ID":VpcId,"Inbound Rules":IpPermissions[*].{Protocol:IpProtocol,Port:FromPort,Source:IpRanges[0].CidrIp},"Outbound Rules":IpPermissionsEgress[*].{Protocol:IpProtocol,Port:FromPort,Destination:IpRanges[0].CidrIp}}'

echo "🔍 Security Group Validation:"
echo "✓ PostgreSQL port (5432) is restricted to VPC CIDR range"
echo "✓ No public internet access"
echo "✓ Proper tagging applied"

## Next Steps 🚀

**Create an Aurora Serverless v2 cluster**
   - Configure 0 minimum capacity to allow it scale to 0 automatically when no activity
   - Create RDS proxy with SSL connection and IAM authentication
   - Set appropriate backup retention periods
   - Enable Performance Insights for monitoring
   - Use the S3 endpoint for data import/export operations

Let's go to [2.1.2_create_your_first_aurora_postgresql_part2.ipynb](2.1.2_create_your_first_aurora_postgresql_part2.ipynb).

## Cleanup

> ⚠️ **Important**: The following commands will delete all networking resources created in this workshop. You will need VPC resources in [the next section when creating an Aurora cluster](2.1.2_create_your_first_aurora_postgresql_part2.ipynb). Only run these if you want to completely remove the VPC and associated resources.

### Cost Consideration
Running these cleanup commands will help you avoid ongoing charges for:
- VPC resources (free but good practice)
- Any associated resources that might incur costs

> 💡 **Note**: To clean up the VPC launched by the AWS CloudFormation template, then go to the [AWS CloudFormation console](https://console.aws.amazon.com/cloudformation/home/), select the AWS CloudFormation stack, and delete it.

Resources must be deleted in the correct order due to dependencies:

In [ ]:
%%bash
echo "🗑️ Starting cleanup process..."

# Source variables if available
if [ -f .vpc_vars ]; then
    source .vpc_vars
else
    echo "Variables file not found, attempting to discover resources..."
    VPC_ID=$(aws ec2 describe-vpcs --filters "Name=tag:Name,Values=aurora-vpc" --query 'Vpcs[0].VpcId' --output text)
    SG_ID=$(aws ec2 describe-security-groups --filters "Name=tag:Name,Values=aurora-sg" --query 'SecurityGroups[0].GroupId' --output text)
fi

if [ "$VPC_ID" == "None" ] || [ -z "$VPC_ID" ]; then
    echo "No VPC found to clean up"
    exit 0
fi

echo "Cleaning up VPC: $VPC_ID"

# 1. Delete S3 VPC Endpoint
echo "Deleting S3 VPC Endpoint..."
S3_ENDPOINT_ID=$(aws ec2 describe-vpc-endpoints --filters "Name=vpc-id,Values=$VPC_ID" "Name=service-name,Values=com.amazonaws.*.s3" --query 'VpcEndpoints[0].VpcEndpointId' --output text)
if [ "$S3_ENDPOINT_ID" != "None" ] && [ ! -z "$S3_ENDPOINT_ID" ]; then
    aws ec2 delete-vpc-endpoint --vpc-endpoint-id $S3_ENDPOINT_ID 2>/dev/null || echo "S3 endpoint not found"
fi

# 2. Delete Security Group
echo "Deleting Security Group..."
if [ "$SG_ID" != "None" ] && [ ! -z "$SG_ID" ]; then
    aws ec2 delete-security-group --group-id $SG_ID 2>/dev/null || echo "Security group not found"
fi

# 3. Disassociate and Delete Route Tables
echo "Deleting route tables..."
ROUTE_TABLES=$(aws ec2 describe-route-tables --filters "Name=vpc-id,Values=$VPC_ID" "Name=tag:Name,Values=aurora-*" --query 'RouteTables[*].RouteTableId' --output text)
for RTB in $ROUTE_TABLES; do
    # Disassociate subnets first
    ASSOCIATIONS=$(aws ec2 describe-route-tables --route-table-ids $RTB --query 'RouteTables[0].Associations[?Main==`false`].RouteTableAssociationId' --output text)
    for ASSOC in $ASSOCIATIONS; do
        aws ec2 disassociate-route-table --association-id $ASSOC 2>/dev/null
    done
    aws ec2 delete-route-table --route-table-id $RTB 2>/dev/null || echo "Route table $RTB not found"
done

# 4. Delete Subnets
echo "Deleting subnets..."
SUBNETS=$(aws ec2 describe-subnets --filters "Name=vpc-id,Values=$VPC_ID" --query 'Subnets[*].SubnetId' --output text)
for SUBNET in $SUBNETS; do
    aws ec2 delete-subnet --subnet-id $SUBNET 2>/dev/null || echo "Subnet $SUBNET not found"
done

# 5. Detach and Delete Internet Gateway
echo "Deleting Internet Gateway..."
IGW_ID=$(aws ec2 describe-internet-gateways --filters "Name=attachment.vpc-id,Values=$VPC_ID" --query 'InternetGateways[0].InternetGatewayId' --output text)
if [ "$IGW_ID" != "None" ] && [ ! -z "$IGW_ID" ]; then
    aws ec2 detach-internet-gateway --internet-gateway-id $IGW_ID --vpc-id $VPC_ID 2>/dev/null
    aws ec2 delete-internet-gateway --internet-gateway-id $IGW_ID 2>/dev/null || echo "Internet gateway not found"
fi

# 6. Delete VPC
echo "Deleting VPC..."
aws ec2 delete-vpc --vpc-id $VPC_ID 2>/dev/null || echo "VPC not found"

echo "✅ Cleanup completed. All networking resources have been deleted."

This step removes temporary files created during the workshop.

In [ ]:
%%bash
# Cleanup temporary files
rm -f .vpc_vars
echo "✅ Temporary files cleaned up"

## Additional Resources 📚

### Aurora Best Practices
- [Aurora Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)
- [Performance Insights Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)
- [Serverless v2 Capacity Management](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/aurora-serverless-v2.html)

### Data Import/Export
- [Importing Data from S3 to Aurora PostgreSQL](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/postgresql-s3-import.html)
- [Exporting Data from Aurora PostgreSQL to S3](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/postgresql-s3-export.html)

### Networking & Security
- [VPC Security Best Practices](https://docs.aws.amazon.com/vpc/latest/userguide/vpc-security-best-practices.html)
- [VPC Endpoints for S3](https://docs.aws.amazon.com/vpc/latest/privatelink/vpc-endpoints-s3.html)